In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import recall_score
from PIL import Image

In [2]:
# 데이터셋 클래스 정의
class FoodDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
# recall 계산 함수
def calculate_recall(preds, labels):
    preds = torch.sigmoid(preds).cpu().numpy()
    preds = np.round(preds)
    labels = labels.cpu().numpy()
    return recall_score(labels, preds, zero_division=0, average='macro')

In [11]:
# 데이터 불러오기
train_df = pd.read_csv('train_data_set_food.csv')
test_df = pd.read_csv('test_data_set_food.csv')

# 재료 목록 정의
ingredients = ['chicken', 'shrimp', 'cheese', 'pork', 'cream', 'tofu', 'lobster', 'peanut', 'bread', 'crab', 'egg']

for ingredient in ingredients:
    # 데이터 선택 및 라벨링
    positive_train_df = train_df[train_df[ingredient] == 1]
    negative_train_df = train_df[train_df[ingredient] == 0].sample(n=len(positive_train_df), random_state=42)
    positive_test_df = test_df[test_df[ingredient] == 1]
    negative_test_df = test_df[test_df[ingredient] == 0].sample(n=len(positive_test_df), random_state=42)

    balanced_train_df = pd.concat([positive_train_df, negative_train_df])
    balanced_test_df = pd.concat([positive_test_df, negative_test_df])

    balanced_train_df['label'] = balanced_train_df[ingredient]
    balanced_test_df['label'] = balanced_test_df[ingredient]

    # 이미지 경로 설정
    balanced_train_df['image_path'] = balanced_train_df.apply(lambda row: os.path.join('food_dataset/images', row['food_class'], 'png', row['image_name']), axis=1)
    balanced_test_df['image_path'] = balanced_test_df.apply(lambda row: os.path.join('food_dataset/images', row['food_class'], 'png', row['image_name']), axis=1)

    # 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # 데이터셋 객체 생성 및 DataLoader 정의
    train_dataset = FoodDataset(balanced_train_df, transform=transform)
    test_dataset = FoodDataset(balanced_test_df, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # 모델 초기화
    # 모델 초기화
    weights = ResNet18_Weights.IMAGENET1K_V1 if torch.cuda.is_available() else ResNet18_Weights.DEFAULT
    model = models.resnet18(weights=weights)
    model.fc = nn.Linear(model.fc.in_features, 1)  # 이진 분류를 위한 출력 뉴런 1개
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    
    # 학습 파라미터
    epochs = 10
    
    # 모델 학습 및 평가
    for epoch in range(epochs):
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            labels = labels.unsqueeze(1).float()  # 레이블 차원 변경
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if (i + 1) % 10 == 0:
                print(f'Epoch {epoch+1}, Batch {i+1}, Training Loss: {loss.item()}')

        # 모델 평가
        model.eval()
        total_recall = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                recall = calculate_recall(outputs, labels)
                total_recall += recall

        average_recall = total_recall / len(test_loader)
        print(f'Epoch {epoch+1}, Average Recall: {average_recall}')

        # 모델 저장
    torch.save(model.state_dict(), f'new_{ingredient}_model.pth')

Epoch 1, Batch 10, Training Loss: 0.677469789981842
Epoch 1, Batch 20, Training Loss: 0.6543619632720947
Epoch 1, Batch 30, Training Loss: 0.6065874099731445
Epoch 1, Batch 40, Training Loss: 0.6472185850143433
Epoch 1, Batch 50, Training Loss: 0.6304143071174622
Epoch 1, Batch 60, Training Loss: 0.632469117641449
Epoch 1, Batch 70, Training Loss: 0.6146209239959717
Epoch 1, Batch 80, Training Loss: 0.5429264903068542
Epoch 1, Batch 90, Training Loss: 0.6830814480781555
Epoch 1, Batch 100, Training Loss: 0.6189112067222595
Epoch 1, Batch 110, Training Loss: 0.5077006816864014
Epoch 1, Average Recall: 0.36679638610763454
Epoch 2, Batch 10, Training Loss: 0.504433274269104
Epoch 2, Batch 20, Training Loss: 0.47505462169647217
Epoch 2, Batch 30, Training Loss: 0.4459352493286133
Epoch 2, Batch 40, Training Loss: 0.40818148851394653
Epoch 2, Batch 50, Training Loss: 0.5736027956008911
Epoch 2, Batch 60, Training Loss: 0.44962480664253235
Epoch 2, Batch 70, Training Loss: 0.4986337125301361

Epoch 4, Batch 30, Training Loss: 0.3554750680923462
Epoch 4, Batch 40, Training Loss: 0.24224138259887695
Epoch 4, Batch 50, Training Loss: 0.3318103551864624
Epoch 4, Batch 60, Training Loss: 0.3028776943683624
Epoch 4, Batch 70, Training Loss: 0.2829110026359558
Epoch 4, Batch 80, Training Loss: 0.31160834431648254
Epoch 4, Batch 90, Training Loss: 0.30372291803359985
Epoch 4, Batch 100, Training Loss: 0.4254316985607147
Epoch 4, Average Recall: 0.4072516025641026
Epoch 5, Batch 10, Training Loss: 0.24174433946609497
Epoch 5, Batch 20, Training Loss: 0.2711203396320343
Epoch 5, Batch 30, Training Loss: 0.20196621119976044
Epoch 5, Batch 40, Training Loss: 0.34342801570892334
Epoch 5, Batch 50, Training Loss: 0.3275623023509979
Epoch 5, Batch 60, Training Loss: 0.2784552276134491
Epoch 5, Batch 70, Training Loss: 0.25324884057044983
Epoch 5, Batch 80, Training Loss: 0.25918343663215637
Epoch 5, Batch 90, Training Loss: 0.38417255878448486
Epoch 5, Batch 100, Training Loss: 0.35178789

Epoch 3, Batch 240, Training Loss: 0.32409006357192993
Epoch 3, Batch 250, Training Loss: 0.34889715909957886
Epoch 3, Batch 260, Training Loss: 0.39005181193351746
Epoch 3, Batch 270, Training Loss: 0.37744203209877014
Epoch 3, Average Recall: 0.4012985280319024
Epoch 4, Batch 10, Training Loss: 0.3163953423500061
Epoch 4, Batch 20, Training Loss: 0.2840624749660492
Epoch 4, Batch 30, Training Loss: 0.36833927035331726
Epoch 4, Batch 40, Training Loss: 0.2737184166908264
Epoch 4, Batch 50, Training Loss: 0.264776349067688
Epoch 4, Batch 60, Training Loss: 0.4017985761165619
Epoch 4, Batch 70, Training Loss: 0.3202695846557617
Epoch 4, Batch 80, Training Loss: 0.2714241147041321
Epoch 4, Batch 90, Training Loss: 0.2727409899234772
Epoch 4, Batch 100, Training Loss: 0.2741881310939789
Epoch 4, Batch 110, Training Loss: 0.30248063802719116
Epoch 4, Batch 120, Training Loss: 0.33549046516418457
Epoch 4, Batch 130, Training Loss: 0.2994648814201355
Epoch 4, Batch 140, Training Loss: 0.2298

Epoch 9, Batch 80, Training Loss: 0.05887305736541748
Epoch 9, Batch 90, Training Loss: 0.050336845219135284
Epoch 9, Batch 100, Training Loss: 0.03883109986782074
Epoch 9, Batch 110, Training Loss: 0.05147847160696983
Epoch 9, Batch 120, Training Loss: 0.030235782265663147
Epoch 9, Batch 130, Training Loss: 0.0377049520611763
Epoch 9, Batch 140, Training Loss: 0.06961099058389664
Epoch 9, Batch 150, Training Loss: 0.04222485423088074
Epoch 9, Batch 160, Training Loss: 0.033776216208934784
Epoch 9, Batch 170, Training Loss: 0.05822237208485603
Epoch 9, Batch 180, Training Loss: 0.02958717942237854
Epoch 9, Batch 190, Training Loss: 0.0752926766872406
Epoch 9, Batch 200, Training Loss: 0.0662352442741394
Epoch 9, Batch 210, Training Loss: 0.06283382326364517
Epoch 9, Batch 220, Training Loss: 0.06664320826530457
Epoch 9, Batch 230, Training Loss: 0.036932073533535004
Epoch 9, Batch 240, Training Loss: 0.04118943586945534
Epoch 9, Batch 250, Training Loss: 0.03844171389937401
Epoch 9, Ba

Epoch 5, Batch 160, Training Loss: 0.28219932317733765
Epoch 5, Batch 170, Training Loss: 0.161311537027359
Epoch 5, Batch 180, Training Loss: 0.2506256401538849
Epoch 5, Batch 190, Training Loss: 0.25798314809799194
Epoch 5, Batch 200, Training Loss: 0.26434579491615295
Epoch 5, Batch 210, Training Loss: 0.2656509280204773
Epoch 5, Average Recall: 0.38799185463659147
Epoch 6, Batch 10, Training Loss: 0.3041685223579407
Epoch 6, Batch 20, Training Loss: 0.1893516182899475
Epoch 6, Batch 30, Training Loss: 0.23082883656024933
Epoch 6, Batch 40, Training Loss: 0.1848507523536682
Epoch 6, Batch 50, Training Loss: 0.1671205312013626
Epoch 6, Batch 60, Training Loss: 0.16614185273647308
Epoch 6, Batch 70, Training Loss: 0.21429775655269623
Epoch 6, Batch 80, Training Loss: 0.1672828495502472
Epoch 6, Batch 90, Training Loss: 0.19265642762184143
Epoch 6, Batch 100, Training Loss: 0.1375107765197754
Epoch 6, Batch 110, Training Loss: 0.14048129320144653
Epoch 6, Batch 120, Training Loss: 0.18

...

Epoch 2, Batch 470, Training Loss: 0.41293787956237793
Epoch 2, Batch 480, Training Loss: 0.234572172164917
Epoch 2, Batch 490, Training Loss: 0.42211195826530457
Epoch 2, Batch 500, Training Loss: 0.31346461176872253
Epoch 2, Average Recall: 0.42579394848712177
Epoch 3, Batch 10, Training Loss: 0.25363510847091675
Epoch 3, Batch 20, Training Loss: 0.23124322295188904
Epoch 3, Batch 30, Training Loss: 0.21626052260398865
Epoch 3, Batch 40, Training Loss: 0.24553976953029633
Epoch 3, Batch 50, Training Loss: 0.20845451951026917
Epoch 3, Batch 60, Training Loss: 0.39466428756713867
Epoch 3, Batch 70, Training Loss: 0.3123512864112854
Epoch 3, Batch 80, Training Loss: 0.1938730627298355
Epoch 3, Batch 90, Training Loss: 0.2716210186481476
Epoch 3, Batch 100, Training Loss: 0.2948238253593445
Epoch 3, Batch 110, Training Loss: 0.28519406914711
Epoch 3, Batch 120, Training Loss: 0.26350945234298706
Epoch 3, Batch 130, Training Loss: 0.2423400729894638
Epoch 3, Batch 140, Training Loss: 0.26

Epoch 5, Batch 460, Training Loss: 0.11313357204198837
Epoch 5, Batch 470, Training Loss: 0.11629787087440491
Epoch 5, Batch 480, Training Loss: 0.09969624131917953
Epoch 5, Batch 490, Training Loss: 0.10199227929115295
Epoch 5, Batch 500, Training Loss: 0.1620100736618042
Epoch 5, Average Recall: 0.43263940985246313
Epoch 6, Batch 10, Training Loss: 0.05287185311317444
Epoch 6, Batch 20, Training Loss: 0.06785270571708679
Epoch 6, Batch 30, Training Loss: 0.15322494506835938
Epoch 6, Batch 40, Training Loss: 0.12365461885929108
Epoch 6, Batch 50, Training Loss: 0.047182731330394745
Epoch 6, Batch 60, Training Loss: 0.059053897857666016
Epoch 6, Batch 70, Training Loss: 0.034851688891649246
Epoch 6, Batch 80, Training Loss: 0.11010454595088959
Epoch 6, Batch 90, Training Loss: 0.05460668355226517
Epoch 6, Batch 100, Training Loss: 0.12236916273832321
Epoch 6, Batch 110, Training Loss: 0.0497581921517849
Epoch 6, Batch 120, Training Loss: 0.09251844137907028
Epoch 6, Batch 130, Training

Epoch 8, Batch 430, Training Loss: 0.08608651906251907
Epoch 8, Batch 440, Training Loss: 0.01300623919814825
Epoch 8, Batch 450, Training Loss: 0.029971685260534286
Epoch 8, Batch 460, Training Loss: 0.013588616624474525
Epoch 8, Batch 470, Training Loss: 0.022155288606882095
Epoch 8, Batch 480, Training Loss: 0.10222416371107101
Epoch 8, Batch 490, Training Loss: 0.01948484033346176
Epoch 8, Batch 500, Training Loss: 0.021339159458875656
Epoch 8, Average Recall: 0.4328210958989748
Epoch 9, Batch 10, Training Loss: 0.009664938785135746
Epoch 9, Batch 20, Training Loss: 0.0141132278367877
Epoch 9, Batch 30, Training Loss: 0.01643122360110283
Epoch 9, Batch 40, Training Loss: 0.03791600465774536
Epoch 9, Batch 50, Training Loss: 0.011583283543586731
Epoch 9, Batch 60, Training Loss: 0.011461431160569191
Epoch 9, Batch 70, Training Loss: 0.0613429918885231
Epoch 9, Batch 80, Training Loss: 0.016065236181020737
Epoch 9, Batch 90, Training Loss: 0.007861841470003128
Epoch 9, Batch 100, Tra

Epoch 1, Batch 10, Training Loss: 0.744601309299469
Epoch 1, Batch 20, Training Loss: 0.7018846869468689
Epoch 1, Batch 30, Training Loss: 0.7240990400314331
Epoch 1, Batch 40, Training Loss: 0.6189072728157043
Epoch 1, Batch 50, Training Loss: 0.6541412472724915
Epoch 1, Batch 60, Training Loss: 0.6191960573196411
Epoch 1, Batch 70, Training Loss: 0.61759352684021
Epoch 1, Batch 80, Training Loss: 0.6121242046356201
Epoch 1, Batch 90, Training Loss: 0.5798351168632507
Epoch 1, Batch 100, Training Loss: 0.6163702011108398
Epoch 1, Batch 110, Training Loss: 0.5497775673866272
Epoch 1, Batch 120, Training Loss: 0.6139779686927795
Epoch 1, Batch 130, Training Loss: 0.49913084506988525
Epoch 1, Batch 140, Training Loss: 0.5004692077636719
Epoch 1, Batch 150, Training Loss: 0.5781500935554504
Epoch 1, Batch 160, Training Loss: 0.5666325688362122
Epoch 1, Batch 170, Training Loss: 0.4382256865501404
Epoch 1, Batch 180, Training Loss: 0.5446555614471436
Epoch 1, Batch 190, Training Loss: 0.64

Epoch 6, Batch 240, Training Loss: 0.13768932223320007
Epoch 6, Batch 250, Training Loss: 0.14043350517749786
Epoch 6, Average Recall: 0.4152968287674727
Epoch 7, Batch 10, Training Loss: 0.09865707904100418
Epoch 7, Batch 20, Training Loss: 0.06904307007789612
Epoch 7, Batch 30, Training Loss: 0.095075324177742
Epoch 7, Batch 40, Training Loss: 0.1469830870628357
Epoch 7, Batch 50, Training Loss: 0.11210278421640396
Epoch 7, Batch 60, Training Loss: 0.07636594772338867
Epoch 7, Batch 70, Training Loss: 0.09504079073667526
Epoch 7, Batch 80, Training Loss: 0.10471309721469879
Epoch 7, Batch 90, Training Loss: 0.06993618607521057
Epoch 7, Batch 100, Training Loss: 0.1793447881937027
Epoch 7, Batch 110, Training Loss: 0.10395123064517975
Epoch 7, Batch 120, Training Loss: 0.09533260017633438
Epoch 7, Batch 130, Training Loss: 0.07118261605501175
Epoch 7, Batch 140, Training Loss: 0.08060947060585022
Epoch 7, Batch 150, Training Loss: 0.14614491164684296
Epoch 7, Batch 160, Training Loss: